In [1]:
%load_ext dotenv
%dotenv

In [5]:
import os
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
from datetime import timedelta

import seaborn as sns
import matplotlib.pyplot as plt



In [3]:
df_al = pd.read_csv('../data/aller_from_paris.csv', index_col=0)
df_re = pd.read_csv('../data/retour_to_paris.csv', index_col=0)

In [6]:
def compute_travel_time(df):
    df['datetime_depart'] = df['date'] + ' ' + df['heure_depart']
    df['datetime_arrivee'] = df['date'] + ' ' + df['heure_arrivee']
    df['datetime_depart'] = pd.to_datetime(df['datetime_depart'])
    df['datetime_arrivee'] = pd.to_datetime(df['datetime_arrivee'])

    df['temps_trajet'] = df['datetime_arrivee'] - df['datetime_depart']

    df['temps_trajet'] = df['temps_trajet'].dt.total_seconds() // 60
    df.loc[df['temps_trajet'] < 0,'datetime_arrivee'] = df.loc[df['temps_trajet'] < 0,'datetime_arrivee'] + timedelta(days = 1)

    df['temps_trajet'] = df['datetime_arrivee'] - df['datetime_depart']
    df['temps_trajet'] = df['temps_trajet'].dt.total_seconds() // 60

    return df

In [7]:
df_al = compute_travel_time(df_al)
df_re = compute_travel_time(df_re)

# Trip

In [8]:
date_depart = '2022-04-05'
date_retour = '2022-04-08'

In [ ]:
df_al = df_al.set_index('datetime_depart').copy()
df_re = df_re.set_index('datetime_depart').copy()


In [26]:
def filter_datetime(df, date, hour, kind = 'before'):
    temp = df.loc[date]
    if kind == 'before':
        return temp[temp.index.hour < hour]
    elif kind == 'after':
        return temp[temp.index.hour > hour]
    else:
        return None


In [28]:
poss_al = filter_datetime(df_al, date_depart, 8)
poss_re = filter_datetime(df_re, date_retour, 16, kind = 'after')

In [33]:
cities_ok = list(set(poss_al.destination) & set(poss_re.origine))

In [34]:
print(cities_ok)

['STRASBOURG', 'NIORT', 'TOURS', 'DAX', 'BETHUNE', 'CHATEAUROUX', 'GRENOBLE', 'TARBES', 'BORDEAUX ST JEAN', 'HAZEBROUCK', 'DOUAI', 'CALAIS FRETHUN', 'MONTBARD', 'ANGOULEME', 'AGDE', 'LA ROCHELLE VILLE', 'LIMOGES BENEDICTINS', 'SETE', 'MARSEILLE ST CHARLES', 'NARBONNE', 'VALENCE TGV RHONE ALPES SUD', 'Luxembourg', 'MONTAUBAN VILLE BOURBON', 'VALENCIENNES', 'SURGERES', 'PERPIGNAN', 'ANNECY', 'LYON (gares intramuros)', 'METZ VILLE', 'BOULOGNE VILLE', 'TOULOUSE MATABIAU', 'BIARRITZ', 'LOURDES', 'UZERCHE', 'NANCY', 'GOURDON', 'BESANCON VIOTTE', 'MONTPELLIER SUD DE FRANCE', 'BAYONNE', 'LILLE FLANDRES', 'LENS', 'LE CREUSOT MONTCEAU MONTCHANIN', 'ARRAS', 'LE MANS', 'CHAMBERY CHALLES LES EAUX', 'HENDAYE', 'CORBIERES VIERZON VILLE', 'SOUILLAC', 'AIX EN PROVENCE TGV', 'DUNKERQUE', 'LYON ST EXUPERY', 'CAHORS', 'VENDOME VILLIERS SUR LOIR', 'MACON LOCHE TGV', 'CORBIERES LES AUBRAIS ORLEANS', 'CHAMPAGNE ARDENNE', 'ANGERS ST LAUD', 'THIONVILLE', 'BEZIERS', 'LA SOUTERRAINE', 'POITIERS', 'CHATELLERAULT'

In [4]:
set(df_al.destination.values) - set(df_re.origine.values)

{'LANDERNEAU', 'LUZENAC GARANOU'}